In [209]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shriram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [210]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import pickle
from nltk.tokenize import RegexpTokenizer,word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [211]:
data=pd.read_csv("spam.csv")
dataarray=data.to_numpy()
len(dataarray)

5572

In [212]:
dataarray

array([['ham',
        'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'],
       ['ham', 'Ok lar... Joking wif u oni...'],
       ['spam',
        "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"],
       ...,
       ['ham',
        'Pity, * was in mood for that. So...any other suggestions?'],
       ['ham',
        "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free"],
       ['ham', 'Rofl. Its true to its name']], dtype=object)

In [213]:
data.head(5)

,Label,EmailText
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


data.columns

In [214]:
data.columns

Index(['Label', 'EmailText'], dtype='object')

In [215]:
data['Label']=data['Label'].map({"ham":0,"spam":1})

In [216]:
data.head()


,Label,EmailText
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [217]:
x=dataarray[:,1]
y=dataarray[:,0]

In [218]:
x.shape,y.shape

((5572,), (5572,))

In [219]:
tokenizer=RegexpTokenizer('\w+')
sw=set(stopwords.words('english'))
ps=PorterStemmer()

In [220]:
def dopreprocess(sample):
    sample=sample.lower()
    tokens=tokenizer.tokenize(sample)
    removed_stopwords=[word for word in tokens if word not in sw]
    stemmed_words=[ps.stem(token)for token in removed_stopwords]
    preprocessed=' '.join(stemmed_words)
    return preprocessed

In [221]:
def preprocessed(document):
    docs=[]
    for doc in document:
        docs.append(dopreprocess(doc))
    return docs

In [222]:
x[:3]

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"],
      dtype=object)

In [223]:
stemmed_doc=preprocessed(x)
#three data samples after preprocessing
stemmed_doc[:3]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri 2 wkli comp win fa cup final tkt 21st may 2005 text fa 87121 receiv entri question std txt rate c appli 08452810075over18']

In [224]:
c=CountVectorizer()

In [225]:
vocab=c.fit_transform(x)

In [226]:
x=vocab.todense()

In [227]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [228]:
x_train.shape

(4457, 8679)

In [229]:
x_test.shape

(1115, 8679)

In [230]:
y_train.shape

(4457,)

In [231]:
model=MultinomialNB()

In [232]:
model.fit(x_train,y_train)

MultinomialNB()

In [233]:
result=model.score(x_test,y_test)
result

0.9802690582959641

In [234]:
result=result*100
result

98.02690582959642

In [235]:
pickle.dump(model,open("spam.pkl","wb"))

In [236]:
pickle.dump(c,open("vectorize.pkl","wb"))

In [237]:
clf=pickle.load(open("spam.pkl","rb"))
clf

MultinomialNB()

In [244]:
def prepare(messages):
    d=preprocessed(messages)
    return c.transform(d)

In [249]:
message="you won/ 200 dollara"
data=[message]
vector=prepare(data)
result=model.predict(vector)
result

array(['spam'], dtype='<U4')